# This is just draft kernel, where I have tested different staff I was interested in trying.
# That's why it looks really trashy, so if you don't want to harm your eyes, then just skip it :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import sys
import random
import warnings
import zipfile

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

## As we have zip files first I need to unzip my folders with images
### Here I specify my pathes to necessary folders

In [ ]:
# Unzipping some files
stage1_train = 'stage1_train'
stage1_test = 'stage1_test'

## Now I will unzip them to the output folder

In [ ]:
# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/data-science-bowl-2018/"+stage1_train+".zip","r") as z:
    z.extractall(stage1_train)

In [ ]:
# Will unzip the files so that you can see them..
with zipfile.ZipFile("../input/data-science-bowl-2018/"+stage1_test+".zip","r") as z:
    z.extractall(stage1_test)

# Now let's specify some constants that we will use 

In [ ]:
TRAIN_PATH = '../working/stage1_train/'
TEST_PATH = '../working/stage1_test/'
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed = seed
np.random.seed = seed

## Getting images' ids

In [ ]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

In [ ]:
import os
import random
import sys
import warnings
import numpy as np
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.utils import Progbar

## Loading data

In [ ]:
def get_train_data(IMG_WIDTH=256, IMG_HEIGHT=256, IMG_CHANNELS=3):
    X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    print('Getting and resizing train images and masks ... ')
    sys.stdout.flush()
    if os.path.isfile('train_img.npy') and os.path.isfile('train_mask.npy'):
        print('Train file loaded from memory')
        X_train = np.load('train_img.npy')
        Y_train = np.load('train_mask.npy')
        return X_train, Y_train
    a = Progbar(len(train_ids))
    for n, id_ in enumerate(train_ids):
        path = TRAIN_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:, :, :IMG_CHANNELS]
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_train[n] = img
        mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
        for mask_file in next(os.walk(path + '/masks/'))[2]:
            mask_ = imread(path + '/masks/' + mask_file)
            mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant',
                                        preserve_range=True), axis=-1)
            mask = np.maximum(mask, mask_)
        Y_train[n] = mask
        a.update(n)
    np.save('train_img', X_train)
    np.save('train_mask', Y_train)
    return X_train, Y_train

In [ ]:
def get_test_data(IMG_WIDTH=256, IMG_HEIGHT=256, IMG_CHANNELS=3):
    X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    sizes_test = []
    print('\nGetting and resizing test images ... ')
    sys.stdout.flush()
    if os.path.isfile('test_img.npy') and os.path.isfile('test_size.npy'):
        print('Test file loaded from memory')
        X_test = np.load('test_img.npy')
        sizes_test = np.load('test_size.npy')
        return X_test,sizes_test
    b = Progbar(len(test_ids))
    for n, id_ in enumerate(test_ids):
        path = TEST_PATH + id_
        img = imread(path + '/images/' + id_ + '.png')[:, :, :IMG_CHANNELS]
        sizes_test.append([img.shape[0], img.shape[1]])
        img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
        X_test[n] = img
        b.update(n)
    np.save('test_img', X_test)
    np.save('test_size', sizes_test)
    return X_test,sizes_test

In [ ]:
# get train_data
train_img, train_mask = get_train_data()

In [ ]:
# get test_data
test_img, test_img_sizes = get_test_data()

# Defining U-Net model and metrics
## As a metric I will use Dice Coefficient

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import backend as K

smooth = 1.


# Metric function
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def make_model(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    return model

In [ ]:
# get u_net model
u_net = make_model()

## Preliminary training on 50 epochs

In [ ]:
print("\nTraining...")
u_net.fit(train_img, train_mask, batch_size=16, epochs=50)

# Predicting masks for test images

In [ ]:
print("Predicting")
# Predict on test data
test_mask = u_net.predict(test_img, verbose=1)

## Obtained mask

In [ ]:
plt.imshow(test_mask[0].reshape(256,256))

In [ ]:
test_img.shape

# Actual image

In [ ]:
plt.imshow(test_img[0])

### Now, as far as I now I can not estimate quality of my masks for test images using Dice Coefficient, because I don't have ground truth masks.
### So what I am going to do, I will split my train set into train/validation set, and use this validation set as test set, predict masks for it, and then evaluate it.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_img, train_mask, test_size=0.15, random_state=2018)

## Defining another model

In [ ]:
# get u_net model
u_net_1 = make_model()

In [ ]:
print("\nTraining...")
u_net_1.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
print("Predicting")
# Predict on test data
test_mask_validation = u_net_1.predict(X_valid, verbose=1)

# Predicted mask

In [ ]:
plt.imshow(test_mask_validation[0].reshape(256,256))

# Ground truth mask

In [ ]:
plt.imshow(y_valid[0].reshape(256,256))

### Now let's evaluate our model

In [ ]:
u_net_1.evaluate(X_valid,y_valid, verbose=1)

In [ ]:
u_net_1.metrics_names

In [ ]:
dice_score = 0
for i in range(len(test_mask_validation)):
    dice_score += dice_coef(test_mask_validation[i].reshape(256,256),y_valid[i].reshape(256,256).astype("float32")).numpy()
print(dice_score/len(test_mask_validation))

# Now let's try adding some early stopping

## Monitoring dice_coef

In [ ]:
callbacks = [
    EarlyStopping(monitor='dice_coef', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='dice_coef', factor=0.1, patience=3, min_lr=0.000001, verbose=1),
    ModelCheckpoint('model_weights_1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
# get u_net model
u_net_es = make_model()

In [ ]:
results = u_net_es.fit(X_train, y_train, batch_size=16, epochs=100, callbacks=callbacks)

# Monitoring loss

In [ ]:
callbacks1 = [
    EarlyStopping(monitor='loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, min_lr=0.000001, verbose=1),
    ModelCheckpoint('model_weights_1.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
# get u_net model
u_net_es_loss = make_model()

In [ ]:
results = u_net_es_loss.fit(X_train, y_train, batch_size=16, epochs=200, callbacks=callbacks1)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["dice_coef"], label="dice_coef")
plt.plot( np.argmin(results.history["dice_coef"]), np.min(results.history["dice_coef"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K


def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def conv2d_block(input_tensor, n_filters, kernel_size=3, dropout=1, activation='relu'):
    # first layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    c1 = Dropout(0.1)(c1)
    # second layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(c1)
    return c1


def unet_model(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3, activation='relu', dropout=0.1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    n_filters = 16
    
    c1 = conv2d_block(s, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    p1 = MaxPooling2D((2, 2), name='pool1')(c1)
    
    
    c2 = conv2d_block(p1, 32, kernel_size=3, dropout=dropout*1, activation=activation)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv2d_block(p2, 64, kernel_size=3, dropout=dropout*2, activation=activation)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv2d_block(p3, 128, kernel_size=3, dropout=dropout*2, activation=activation)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv2d_block(p4, 256, kernel_size=3, dropout=dropout*3, activation=activation)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block(u6, 128, kernel_size=3, dropout=dropout*2, activation=activation)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block(u7, 64, kernel_size=3, dropout=dropout*2, activation=activation)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block(u8, 32, kernel_size=3, dropout=dropout*1, activation=activation)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block(u9, 16, kernel_size=3, dropout=dropout*1, activation=activation)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    return model

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(train_img, train_mask, test_size=0.15, random_state=2018)

## Testing our model with different activation functions 

In [ ]:
# get u_net model
unet_relu = unet_model(activation="relu")

In [ ]:
print("\nTraining...")
unet_relu.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
unet_relu.evaluate(X_valid,y_valid, verbose=1)

# Elu activation

In [ ]:
# get u_net model
unet_elu = unet_model(activation="elu")

In [ ]:
print("\nTraining...")
unet_elu.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
unet_elu.evaluate(X_valid,y_valid, verbose=1)

# Trying adding validation during learning

In [ ]:
# get u_net model
unet_elu_valid = unet_model(activation="elu")

In [ ]:
unet_elu_valid.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_valid, y_valid))

# Adding callbacks

In [ ]:
callbacks_new = [
    EarlyStopping(monitor='dice_coef', mode='max', patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    ModelCheckpoint('model_weights.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

In [ ]:
# get u_net model
unet_elu_call = unet_model(activation="elu")

In [ ]:
unet_elu_call.fit(X_train, y_train, batch_size=16, epochs=50, 
                  validation_data=(X_valid, y_valid), callbacks=callbacks_new)

# Sigmoid activation

In [ ]:
# get u_net model
unet_sigmoid = unet_model(activation="sigmoid")

In [ ]:
print("\nTraining...")
unet_sigmoid.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
unet_sigmoid.evaluate(X_valid,y_valid, verbose=1)

## Best accuracy is obtainde using elu activation function

## Now let's test different droput values

In [ ]:
# get u_net model
unet_elu_02_dropout = unet_model(activation="elu", dropout=0.2)

In [ ]:
print("\nTraining...")
unet_elu_02_dropout.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
# get u_net model
unet_elu_015_dropout = unet_model(activation="elu", dropout=0.15)

In [ ]:
print("\nTraining...")
unet_elu_015_dropout.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
unet_elu_015_dropout.evaluate(X_valid,y_valid, verbose=1)

## trying another activation functions

In [ ]:
# get u_net model
unet_selu_015_dropout = unet_model(activation="selu", dropout=0.15)

In [ ]:
print("\nTraining...")
unet_selu_015_dropout.fit(X_train, y_train, batch_size=16, epochs=50)

# Adding learning rate change over time

In [ ]:
callbacks2 = [
    ReduceLROnPlateau(monitor='loss', factor=0.1, patience=3, min_lr=0.000001, verbose=1),
]

In [ ]:
# get u_net model
unet_elu_015_dropout_cbs = unet_model(activation="elu", dropout=0.15)

In [ ]:
unet_elu_015_dropout_cbs.fit(X_train, y_train, batch_size=16, epochs=50, callbacks=callbacks2)

In [ ]:
from keras

In [ ]:
def conv2d_block_ccd(input_tensor, n_filters, kernel_size=3, dropout=1, activation='relu'):
    # first layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    # second layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(c1)
    c1 = Dropout(0.1)(c1)
    return c1


def unet_model_ccd(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3, activation='relu', dropout=0.1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    n_filters = 16
    
    c1 = conv2d_block_ccd(s, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    p1 = MaxPooling2D((2, 2), name='pool1')(c1)
    
    
    c2 = conv2d_block_ccd(p1, 32, kernel_size=3, dropout=dropout*1, activation=activation)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv2d_block_ccd(p2, 64, kernel_size=3, dropout=dropout*2, activation=activation)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv2d_block_ccd(p3, 128, kernel_size=3, dropout=dropout*2, activation=activation)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv2d_block_ccd(p4, 256, kernel_size=3, dropout=dropout*3, activation=activation)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block_ccd(u6, 128, kernel_size=3, dropout=dropout*2, activation=activation)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block_ccd(u7, 64, kernel_size=3, dropout=dropout*2, activation=activation)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block_ccd(u8, 32, kernel_size=3, dropout=dropout*1, activation=activation)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block_ccd(u9, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    
    d10 = Dropout

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    return model

In [ ]:
# get u_net model
unet_elu_015_dropout_ccd = unet_model_ccd(activation="elu", dropout=0.15)

In [ ]:
unet_elu_015_dropout_ccd.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
from keras.layers.core import Dense

In [ ]:
def unet_model_ccd_d(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3, activation='relu', dropout=0.1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    n_filters = 16
    
    c1 = conv2d_block_ccd(s, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    p1 = MaxPooling2D((2, 2), name='pool1')(c1)
    
    
    c2 = conv2d_block_ccd(p1, 32, kernel_size=3, dropout=dropout*1, activation=activation)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv2d_block_ccd(p2, 64, kernel_size=3, dropout=dropout*2, activation=activation)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv2d_block_ccd(p3, 128, kernel_size=3, dropout=dropout*2, activation=activation)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv2d_block_ccd(p4, 256, kernel_size=3, dropout=dropout*3, activation=activation)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block_ccd(u6, 128, kernel_size=3, dropout=dropout*2, activation=activation)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block_ccd(u7, 64, kernel_size=3, dropout=dropout*2, activation=activation)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block_ccd(u8, 32, kernel_size=3, dropout=dropout*1, activation=activation)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block_ccd(u9, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    
    d10 = Dense(256,activation=activation)(c9)
    d10 = Dense(128,activation=activation)(d10)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(d10)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    return model

In [ ]:
# get u_net model
unet_elu_015_dropout_ccd_dd = unet_model_ccd_d(activation="elu", dropout=0.15)

In [ ]:
unet_elu_015_dropout_ccd_dd.summary()

In [ ]:
unet_elu_015_dropout_ccd_dd.fit(X_train, y_train, batch_size=16, epochs=50)

# Let's try avgpool instead of max pool

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K


def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def conv2d_block(input_tensor, n_filters, kernel_size=3, dropout=1, activation='relu'):
    # first layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    c1 = Dropout(0.1)(c1)
    # second layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(c1)
    return c1


def unet_model_av_pool(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3, activation='relu', dropout=0.1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    n_filters = 16
    
    c1 = conv2d_block(s, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    p1 = AveragePooling2D((2, 2), name='pool1')(c1)
    
    
    c2 = conv2d_block(p1, 32, kernel_size=3, dropout=dropout*1, activation=activation)
    p2 = AveragePooling2D((2, 2))(c2)

    c3 = conv2d_block(p2, 64, kernel_size=3, dropout=dropout*2, activation=activation)
    p3 = AveragePooling2D((2, 2))(c3)

    c4 = conv2d_block(p3, 128, kernel_size=3, dropout=dropout*2, activation=activation)
    p4 = AveragePooling2D((2, 2))(c4)

    c5 = conv2d_block(p4, 256, kernel_size=3, dropout=dropout*3, activation=activation)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block(u6, 128, kernel_size=3, dropout=dropout*2, activation=activation)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block(u7, 64, kernel_size=3, dropout=dropout*2, activation=activation)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block(u8, 32, kernel_size=3, dropout=dropout*1, activation=activation)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block(u9, 16, kernel_size=3, dropout=dropout*1, activation=activation)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    return model

In [ ]:
unet_avpool = unet_model_av_pool(activation='elu')

In [ ]:
unet_avpool.fit(X_train, y_train, batch_size=16, epochs=50)

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K


def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def conv2d_block_nodo(input_tensor, n_filters, kernel_size=3, dropout=1, activation='relu'):
    # first layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    # second layer
    c1 = Conv2D(filters=n_filters, activation=activation, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(c1)
    return c1


def unet_model_nodo(IMG_WIDTH=256,IMG_HEIGHT=256,IMG_CHANNELS=3, activation='relu', dropout=0.1):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = Lambda(lambda x: x / 255)(inputs)
    n_filters = 16
    
    c1 = conv2d_block_nodo(s, 16, kernel_size=3, dropout=dropout*1, activation=activation)
    p1 = MaxPooling2D((2, 2), name='pool1')(c1)
    
    
    c2 = conv2d_block_nodo(p1, 32, kernel_size=3, dropout=dropout*1, activation=activation)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv2d_block_nodo(p2, 64, kernel_size=3, dropout=dropout*2, activation=activation)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = conv2d_block_nodo(p3, 128, kernel_size=3, dropout=dropout*2, activation=activation)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = conv2d_block_nodo(p4, 256, kernel_size=3, dropout=dropout*3, activation=activation)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = conv2d_block_nodo(u6, 128, kernel_size=3, dropout=dropout*2, activation=activation)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = conv2d_block_nodo(u7, 64, kernel_size=3, dropout=dropout*2, activation=activation)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = conv2d_block_nodo(u8, 32, kernel_size=3, dropout=dropout*1, activation=activation)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv2d_block_nodo(u9, 16, kernel_size=3, dropout=dropout*1, activation=activation)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
    return model

In [ ]:
unet_nodo = unet_model_nodo(activation='elu')

In [ ]:
unet_nodo.fit(X_train, y_train, batch_size=16, epochs=50)

# Trying to load model

In [ ]:
from keras.models import load_model

In [ ]:
loaded_model = unet_model(activation="elu")

In [ ]:
loaded_model.load_weights("model_weights.h5")

In [ ]:
predicted_masks = loaded_model.predict(test_img)

In [ ]:
plt.imshow(predicted_masks[0].reshape(256, 256))

In [ ]:
plt.imshow(test_img[0])